# Database: esercizi

## Introduzione

## Connessione in SQLStudio


TODO Connessione in SQLStudio 

Cominciamo a guardare una tabella semplice come `Album`. 

**DA FARE**: Prima di procedere, in SQLiteStudio, nel menu a sinistra sotto il nodo `Tables` fai doppio click sulla tabella `Album` e poi nel pannello principale a destra seleziona la tab 'Data'. 

Adesso nel pannello principale a destra seleziona la tab 'Data': 

![](album-data.png)

Vediamo che ci sono 3 colonne, due con numeri `AlbumId` e `ArtistId` e una di stringhe, chiamata `Title` 

**NOTA**: I nomi delle colonne in SQL possono arbitrariamente scelte da chi crea i database. Quindi non è strettamente necessario che i nomi delle colonne numeriche terminino con `Id`. 

## Connessione in Python

Proviamo adesso a recuperare gli stessi dati della tabella `Album` in Python. SQLite è talmente popolare che la libreria per accederlo viene fornita direttamente con Python, quindi non ci servirà installare niente di particolare e possiamo tuffarci subito nel codice: 

In [1]:
import sqlite3

conn = sqlite3.connect('file:chinook.sqlite?mode=rw', uri=True)


L'operazione qua sopra crea un oggetto connessione e lo assegna alla variabile `conn`.
A cosa ci connettiamo? Ad un database indicato dalla uri `file:chinook.sqlite?mode=rw`.
Cos'è una URI? E' una stringa che denota una locazione da qualche parte, potrebbe essere un database accessibile come servizio via internet o un file sul nostro disco. Nel nostro caso vogliamo indicare un database che abbiamo su disco, perciò useremo il protocollo `file:`

SQLite andrà quindi a cercarsi su disco il file `chinook.sqlite`, nella stessa cartella dove stiamo eseguendo Jupyter. Se il file fosse in qualche sottodirectory, potremmo scrivere per es. 'qualche/cartella/chinook.sqlite'

**NOTA 1**: ci stiamo connettendo al database in formato binario .sqlite , NON al file di testo .sql !

**NOTA 2**: stiamo specificando che lo vogliamo aprire in modalità `mode=rw`, cioè di lettura + scrittura (Read Write). SE il database non esiste, questa funzione lancerà un errore.

**NOTA 3**: se volessimo creare un nuovo database, dovremmo usare la modalità lettura + scrittura + creazione (Read Write Creation), specificando come parametro `mode=rwc` (notare la `c` in più)

**NOTA 4**: in tanti sistemi di database (SQLite incluso), di default quando ci si connette ad un database su disco non esistente, ne creano uno. questo è causa di tantissime imprecazioni, perchè se si sbaglia a scrivere il nome del database
non saranno segnalati errori e ci si ritroverà connessi ad un database vuoto, chiedendosi che fine abbiano fatto i dati. E ci si troverà anche il disco pieno di file di database con nomi sbagliati! 

Tramite l'oggetto connessione `conn` possiamo creare un cosiddetto cursore, che ci consentintirà di eseguire query verso il database. Usare una connessione per fare query equivale a chiedere una risorsa del sistema a Python. Le regole di buona educazione ci dicono che quando chiediamo in prestito qualcosa, dopo averlo usato lo si resituisce. La 'restituzione' equivarrebbe in Python a _chiudere_ la risorsa aperta. Ma mentre usiamo lla risorsa si potrebbe verificare un errore, che potrebbe impedirci di chiudere la risorsa correttamente. Per indicare a Pyhton che vogliamo che la risorsa venga chiusa automaticamente in caso di errore, usiamo il comando `with` come abbiamo fatto per i file:

In [2]:
with conn:                       # col blocco with ci cauteliamo da errori imprevisti
    cursore = conn.cursor()      # otteniamo il cursore
    cursore.execute("SELECT * FROM Album LIMIT 5")       # eseguiamo una query in linguaggio SQL al database 
                                                         # notare che execute di per sè non ritorna 
    
    for riga in cursore.fetchall():       # cursore fetchall() genera una sequenza di righe di risultato della query. 
                                          # in sequenza, le righe una alla volta vengono assegnate'oggetto `riga`
        print(riga)                       # stampiamo la riga ottenuta

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)


Finalmente abbiamo ottenuto la lista delle prime 5 righe dal database per la tabella `Album`.

**DA FARE**: prova a scrivere qua sotto le istruzioni per stampare direttamente tutto risultato di `curesore.fetchall()` . Qual'è il tipo di oggetto che ottieni?

In [3]:
# scrivi il codice

**SOLUZIONE**

```
.
.
.
.
.
.
```
E' una lista Python:

In [4]:
with conn:                       
    cursore = conn.cursor()      
    cursore.execute("SELECT * FROM Album LIMIT 5")                                                                
    print(cursore.fetchall())
    print(type(cursore.fetchall()))

[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]
<class 'list'>


E se volessimo passare agevolmente dei parametri alla query, come per esempio il numero dei risultati da ottenere? Per fare ciò possiamo usare dei cosiddetti _placeholder_, cioè dei caratteri punto di domanda `?` che segnano dove vorremmo mettere le variabili. In questo caso sistituiremo il `5` con un punto di domanda, e passeremo il `5` in una lista di parametri a parte: 

In [5]:
with conn:                       # col blocco with ci cauteliamo da errori imprevisti
    cursore = conn.cursor()      # otteniamo il cursore
    cursore.execute("SELECT * FROM Album LIMIT ?", [5]) # eseguiamo una query in linguaggio SQL al database 
                                                         # notare che execute di per sè non ritorna 
    
    for riga in cursore.fetchall():       # cursore fetchall() genera una sequenza di righe di risultato della query. 
                                          # in sequenza, le righe una alla volta vengono assegnate'oggetto `riga`
        print(riga)                       # stampiamo la riga ottenuta

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)


Si possono anche aggiungere più punti di domanda, basta per ognuno passare il corrispondente parametro nella lista: 

In [6]:
with conn:                       # col blocco with ci cauteliamo da errori imprevisti
    cursore = conn.cursor()      # otteniamo il cursore
    cursore.execute("SELECT * FROM Album WHERE AlbumId < ? AND ArtistId < ?", [30,5])
    
    for riga in cursore.fetchall():       # cursore fetchall() genera una sequenza di righe di risultato della query. 
                                          # in sequenza, le righe una alla volta vengono assegnate'oggetto `riga`
        print(riga)                       # stampiamo la riga ottenuta

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)
(6, 'Jagged Little Pill', 4)


## Funzione Esegui Query

Per agevolare le prossime operazioni, ci definiamo una funzione `esegui` che esegue le query che desideriamo e ritorna la lista delle righe ottenute:

In [7]:
def esegui(conn, query, params=()):
    """
    Esegue una query usando la connessione conn, e ritorna la lista di risultati ottenuti.
     
    In params, possiamo mettere una lista di parametri
    con i parametri per la nostra query. 
    """
    with conn:
        cur = conn.cursor()
        cur.execute(query, params)
        return cur.fetchall()

Facciamo una prova: 

In [8]:
import sqlite3

conn = sqlite3.connect('file:chinook.sqlite?mode=rw', uri=True)

esegui(conn, "SELECT * FROM Album LIMIT 5")

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3)]

Proviamo a passare dei parametri: 

In [9]:
esegui(conn, "SELECT * FROM Album WHERE AlbumId < ? AND ArtistId < ?", (30, 5))

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3),
 (6, 'Jagged Little Pill', 4)]

**DA FARE**: In SQLStudio, creare una query che seleziona gli album con id compreso tra 3 e 5 inclusi: 

1) apri il query editor con `Alt+E`
2) immetti la query
3) eseguila premendo F9

**DA FARE**: chiama `esegui` per eseguire la stessa query, usando i parametri

In [10]:
# scrivi il comando

**SOLUZIONE**

```
.
.
.
.
.
.
```

In [11]:
esegui(conn, "SELECT * FROM Album WHERE AlbumId >= ? AND AlbumId <= ?", (3, 5))

[(3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]

### Struttura della tabella

**DA FARE**: Guarda meglio la tab `Structure` di `Album`:

![](album-structure.png)

### DDL

Confronta quanto sopra con la tab `DDL` (Data Definition Language), che contiene le istruzioni SQL per creare la tabella nel database:

![](album-ddl.png)

Una caratteristica che database è la possibilità di dichiarare vincoli sui dati inseriti. Per esempio, qua notiamo che

* la tabella ha una cosiddetta _chiave primaria_ (`PRIMARY KEY`): in essa asserisce che non possono esistere due righe con lo stesso `AlbumId`.

* la tabella definisce la colonna `ArtistId` come _chiave esterna_ (`FOREIGN KEY`), asserendo che ai valori in quella colonna deve sempre corrispondere un id esistente nella colonna `ArtistId` **della tabella `Artist`**. Quindi non ci si potrà mai riferire ad un artista non esistente

**DA FARE**: Vai alla tab `Data` e prova a cambiare un ArtistId mettendo un numero inesistente (tipo 1000). Apparentemente il database non si lamenterà, ma solo perchè al momento non abbiamo ancora registrato il cambiamento su disco, cioè non abbiamo operato una operazione di _commit_. I commit ci permettono di eseguire più operazioni in modo atomico, nel senso che o tutti i cambiamenti fatti vengono registrati con successo, o nessuna modifica viene fatta. Prova ad eseguire un commit premendo il bottoncino verde con la spunta (o premere Ctrl-Return). Che succede? Per riparare al danno fatto, esegui _rollback_ col bottoncino rosso con la x (o premi Ctrl-Backspace).  



### Query ai metadati

Una cosa interessante e a volte utile di molti database SQL è che spesso i metadati sul tipo di tabelle nel database sono salvate essi stessi come tabelle, quindi è possibile eseguire query SQL su questi metadati. Per esempio, con SQLite si può eseguire una query del genere. Non la spieghiamo nel dettaglio, limitandoci a mostrare qualche esempio di utilizzo:

In [12]:
def query_schema(conn, tabella):
    """ Ritorna una stringa con le istruzioni SQL per creare la tabella (senza i dati)  """
    return esegui(conn, "SELECT sql FROM sqlite_master WHERE name = ?", (tabella,))[0][0]

In [13]:
print(query_schema(conn, 'Album'))

CREATE TABLE [Album]
(
    [AlbumId] INTEGER  NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),
    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


## JOIN

Nella tabella `Album` per gli artisti vediamo solo dei numeri. Come possiamo fare una query per vedere anche i nomi degli artisti? Useremo il comando SQL `JOIN` : 

**DA FARE**: Per capire cosa succede, esegui le query in SQLStudio

In [14]:
esegui(conn, "SELECT * FROM Album JOIN Artist WHERE Album.ArtistId = Artist.ArtistId LIMIT 5")

[(1, 'For Those About To Rock We Salute You', 1, 1, 'AC/DC'),
 (2, 'Balls to the Wall', 2, 2, 'Accept'),
 (3, 'Restless and Wild', 2, 2, 'Accept'),
 (4, 'Let There Be Rock', 1, 1, 'AC/DC'),
 (5, 'Big Ones', 3, 3, 'Aerosmith')]

Invece del JOIN possiamo usare una virgola:

In [15]:
esegui(conn, "SELECT * FROM Album, Artist WHERE Album.ArtistId = Artist.ArtistId LIMIT 5")

[(1, 'For Those About To Rock We Salute You', 1, 1, 'AC/DC'),
 (2, 'Balls to the Wall', 2, 2, 'Accept'),
 (3, 'Restless and Wild', 2, 2, 'Accept'),
 (4, 'Let There Be Rock', 1, 1, 'AC/DC'),
 (5, 'Big Ones', 3, 3, 'Aerosmith')]

Meglio ancora, in questo caso visto che abbiamo lo stesso nome di colonna in  entrambe le tabelle, possiamo usare la clausola `USING` che ha anche il beneficio di eliminare la colonna duplicata:

In [16]:
esegui(conn, "SELECT * FROM Album, Artist USING(ArtistId) LIMIT 5")

[(1, 'For Those About To Rock We Salute You', 1, 'AC/DC'),
 (2, 'Balls to the Wall', 2, 'Accept'),
 (3, 'Restless and Wild', 2, 'Accept'),
 (4, 'Let There Be Rock', 1, 'AC/DC'),
 (5, 'Big Ones', 3, 'Aerosmith')]

Infine possiamo filtrare solo le colonne che ci interessano, `Title` di album e `Name` di `Artisti`. Per chiarezza, possiamo identificare le tabelle con variabili che assegnamo in FROM (qua usiamo i nomi `ALB` e `ART` ma potrebbero essere qualsiasi): 

In [17]:
esegui(conn, "SELECT ALB.Title, ART.Name  FROM Album ALB, Artist ART USING(ArtistId) LIMIT 5")

[('For Those About To Rock We Salute You', 'AC/DC'),
 ('Balls to the Wall', 'Accept'),
 ('Restless and Wild', 'Accept'),
 ('Let There Be Rock', 'AC/DC'),
 ('Big Ones', 'Aerosmith')]

## Tabella Track

Passiamo adesso ad una tabella più complessa, come `Track`, che contiene un po' di canzoni ascoltate dagli utenti di iTunes: 

In [18]:
esegui(conn, "SELECT * FROM Track LIMIT 5")

[(1,
  'For Those About To Rock (We Salute You)',
  1,
  1,
  1,
  'Angus Young, Malcolm Young, Brian Johnson',
  343719,
  11170334,
  0.99),
 (2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, 0.99),
 (3,
  'Fast As a Shark',
  3,
  2,
  1,
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman',
  230619,
  3990994,
  0.99),
 (4,
  'Restless and Wild',
  3,
  2,
  1,
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman',
  252051,
  4331779,
  0.99),
 (5,
  'Princess of the Dawn',
  3,
  2,
  1,
  'Deaffy & R.A. Smith-Diesel',
  375418,
  6290521,
  0.99)]

In [19]:
query_schema(conn, "Track")

'CREATE TABLE [Track]\n(\n    [TrackId] INTEGER  NOT NULL,\n    [Name] NVARCHAR(200)  NOT NULL,\n    [AlbumId] INTEGER,\n    [MediaTypeId] INTEGER  NOT NULL,\n    [GenreId] INTEGER,\n    [Composer] NVARCHAR(220),\n    [Milliseconds] INTEGER  NOT NULL,\n    [Bytes] INTEGER,\n    [UnitPrice] NUMERIC(10,2)  NOT NULL,\n    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId]),\n    FOREIGN KEY ([AlbumId]) REFERENCES [Album] ([AlbumId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\n    FOREIGN KEY ([GenreId]) REFERENCES [Genre] ([GenreId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\n    FOREIGN KEY ([MediaTypeId]) REFERENCES [MediaType] ([MediaTypeId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)'

In [20]:
print(query_schema(conn, "Track"))

CREATE TABLE [Track]
(
    [TrackId] INTEGER  NOT NULL,
    [Name] NVARCHAR(200)  NOT NULL,
    [AlbumId] INTEGER,
    [MediaTypeId] INTEGER  NOT NULL,
    [GenreId] INTEGER,
    [Composer] NVARCHAR(220),
    [Milliseconds] INTEGER  NOT NULL,
    [Bytes] INTEGER,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId]),
    FOREIGN KEY ([AlbumId]) REFERENCES [Album] ([AlbumId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([GenreId]) REFERENCES [Genre] ([GenreId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([MediaTypeId]) REFERENCES [MediaType] ([MediaTypeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


In [21]:
esegui(conn, "SELECT Name, Composer FROM Track LIMIT 5")

[('For Those About To Rock (We Salute You)',
  'Angus Young, Malcolm Young, Brian Johnson'),
 ('Balls to the Wall', None),
 ('Fast As a Shark', 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 ('Restless and Wild',
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 ('Princess of the Dawn', 'Deaffy & R.A. Smith-Diesel')]

In [22]:
esegui(conn, "SELECT Name, Composer FROM Track LIMIT 5")[0]

('For Those About To Rock (We Salute You)',
 'Angus Young, Malcolm Young, Brian Johnson')

In [23]:
esegui(conn, "SELECT Name, Composer FROM Track LIMIT 5")[0]

('For Those About To Rock (We Salute You)',
 'Angus Young, Malcolm Young, Brian Johnson')

Per la seconda riga: 

In [24]:
esegui(conn, "SELECT Name, Composer FROM Track LIMIT 5")[1]

('Balls to the Wall', None)

Notiamo che in questo caso manca il compositore. Ma sorge una domanda: nella tabella original SQL, come era segnato il fatto che il compositore mancasse?

**DA FARE**: Usando SQLiteStudio, nel menu a sinistra fai doppio click sulla tabella `Track` e  poi seleziona la tab 'Data' a destra. Scorri le righe finchè non trovi la casella per la colonna `Composer`.

**SOLUZIONE**:

```
.
.  
.
.
.

```

Notiamo che in SQL le caselle vuote si indicano con `NULL`. Dato che `NULL` non è un tipo Python, durante la conversione da oggetti SQL a oggetti Python `NULL` viene convertito nell'oggetto Python `None`.

Proviamo a selezionare dei valori numerici alla nostra query, come per esempio i `Milliseconds`

In [25]:
esegui(conn, "SELECT Name, Milliseconds FROM Track LIMIT 5")

[('For Those About To Rock (We Salute You)', 343719),
 ('Balls to the Wall', 342562),
 ('Fast As a Shark', 230619),
 ('Restless and Wild', 252051),
 ('Princess of the Dawn', 375418)]

In [26]:
esegui(conn, "SELECT Name, Milliseconds FROM Track LIMIT 5")[0]

('For Those About To Rock (We Salute You)', 343719)

In [27]:
esegui(conn, "SELECT Name, Milliseconds FROM Track LIMIT 5")[0][0]

'For Those About To Rock (We Salute You)'

In [28]:
esegui(conn, "SELECT Name, Milliseconds FROM Track LIMIT 5")[0][1]

343719

In [29]:
esegui(conn, "SELECT Name, Milliseconds FROM Track ORDER BY Milliseconds DESC LIMIT 5")

[('Occupation / Precipice', 5286953),
 ('Through a Looking Glass', 5088838),
 ('Greetings from Earth, Pt. 1', 2960293),
 ('The Man With Nine Lives', 2956998),
 ('Battlestar Galactica, Pt. 2', 2956081)]

**DA FARE**: Prova ad usare `ASC` invece di `DESC`

In [30]:
# Scrivi qui la query


**SOLUZIONE**:

```
.
.
.
.
.
```

In [31]:
esegui(conn, "SELECT Name, Composer, Milliseconds FROM Track ORDER BY Milliseconds ASC LIMIT 5")

[('É Uma Partida De Futebol', 'Samuel Rosa', 1071),
 ('Now Sports', None, 4884),
 ('A Statistic', None, 6373),
 ('Oprah', None, 6635),
 ('Commercial 1', 'L. Muggerud', 7941)]

## Pandas

Finora abbiamo usato metodi base di Python, ma ovviamente processare il tutto in pandas è più comodo. 

In [32]:
import pandas

df = pandas.read_sql_query("SELECT Name, Composer, Milliseconds from Track", conn)

In [33]:
df

,Name,Composer,Milliseconds
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719
1,Balls to the Wall,None,342562
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,375418
5,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662
6,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926
7,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834
8,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102
9,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",263497
